# IBM Data Science Course Capstone Project


### Importing libraries

In [1]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

#uncomment and install it, I've already installed 
#!conda install -c conda-forge geopy --yes
!pip install geocoder
from geopy.geocoders import Nominatim 
import geocoder

import requests 
from pandas.io.json import json_normalize

import matplotlib.cm as cm
import matplotlib.colors as colors

from sklearn.cluster import KMeans

#uncomment and install it, I've already installed
#!conda install -c conda-forge folium=0.5.0 --yes 
!pip install folium
import folium

# import the library we use to open URLs
import urllib.request

from bs4 import BeautifulSoup

print('Libraries imported.')


/opt/conda/envs/Python-3.7-main/lib/python3.7/site-packages/secretstorage/dhcrypto.py:16: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes
/opt/conda/envs/Python-3.7-main/lib/python3.7/site-packages/secretstorage/util.py:25: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes
     |████████████████████████████████| 98 kB 7.3 MB/s  eta 0:00:01
/opt/conda/envs/Python-3.7-main/lib/python3.7/site-packages/secretstorage/dhcrypto.py:16: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes
/opt/conda/envs/Python-3.7-main/lib/python3.7/site-packages/secretstorage/util.py:25: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes
     |██████████████████████████

# Scraping wikipedia page of toronto postal codes to acquire location details

In [2]:
url = 'https://en.wikipedia.org/w/index.php?title=List_of_postal_codes_of_Canada:_M&direction=prev&oldid=926287641'

# open the url using urllib.request and put the HTML into the page variable
page = urllib.request.urlopen(url)
soup = BeautifulSoup(page, "lxml")
all_tables = soup.find_all("table")
right_table=soup.find('table', class_='wikitable sortable')
right_table

<table class="wikitable sortable">
<tbody><tr>
<th>Postcode</th>
<th>Borough</th>
<th>Neighbourhood
</th></tr>
<tr>
<td>M1A</td>
<td>Not assigned</td>
<td>Not assigned
</td></tr>
<tr>
<td>M2A</td>
<td>Not assigned</td>
<td>Not assigned
</td></tr>
<tr>
<td>M3A</td>
<td><a href="/wiki/North_York" title="North York">North York</a></td>
<td><a href="/wiki/Parkwoods" title="Parkwoods">Parkwoods</a>
</td></tr>
<tr>
<td>M4A</td>
<td><a href="/wiki/North_York" title="North York">North York</a></td>
<td><a href="/wiki/Victoria_Village" title="Victoria Village">Victoria Village</a>
</td></tr>
<tr>
<td>M5A</td>
<td><a href="/wiki/Downtown_Toronto" title="Downtown Toronto">Downtown Toronto</a></td>
<td><a class="mw-redirect" href="/wiki/Harbourfront_(Toronto)" title="Harbourfront (Toronto)">Harbourfront</a>
</td></tr>
<tr>
<td>M5A</td>
<td><a href="/wiki/Downtown_Toronto" title="Downtown Toronto">Downtown Toronto</a></td>
<td><a href="/wiki/Regent_Park" title="Regent Park">Regent Park</a>
</td></t

# Preparations to convert data into a Dataframe

In [3]:
A=[]
B=[]
C=[]

for row in right_table.findAll('tr'):
    cells=row.findAll('td')
    if len(cells)==3:
        A.append(cells[0].find(text=True))
        B.append(cells[1].find(text=True))
        C.append(cells[2].find(text=True))


In [4]:
df_toronto=pd.DataFrame(A,columns=['PostalCode'])
df_toronto['Borough']=B
df_toronto['Neighborhood']=C
# remove \n values found in PostalCode
df_toronto['PostalCode']=df_toronto['PostalCode'].replace('\n','', regex=True)
df_toronto['Borough']=df_toronto['Borough'].replace('\n','', regex=True)
df_toronto['Neighborhood']=df_toronto['Neighborhood'].replace('\n','', regex=True)
df_toronto.drop(df_toronto[df_toronto['Borough'] == 'Not assigned'].index, axis=0,inplace=True)
print(df_toronto.shape)
df_toronto.head()
df_toronto.reset_index(drop=True).head()
                        


(211, 3)


,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M5A,Downtown Toronto,Regent Park
4,M6A,North York,Lawrence Heights


# Adding Latitude and Longitude values to df_toronto

In [5]:
locgeo_df=pd.read_csv('https://cocl.us/Geospatial_data')
locgeo_df.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [6]:
# Mergin the locgeo_df and df_toronto tables

In [7]:
locgeo_df.rename(columns={'Postal Code':'PostalCode'},inplace=True)
locgeo_df.head()
toronto_data=pd.merge(df_toronto,locgeo_df,on='PostalCode')
print(toronto_data.shape)

(211, 5)


# Acquiring latitude and longitude values of toronto

In [8]:
address='Toronto,Ontario,Canada'
geolocator=Nominatim(user_agent='to_explorer')
location=geolocator.geocode(address)
latitude=location.latitude
longitude=location.longitude
print('The geographical coordinate of Toronto are {},{}'.format(latitude,longitude))

The geographical coordinate of Toronto are 43.6534817,-79.3839347


# Creating a map of Toronto

In [23]:
toronto_map=folium.Map(location=[latitude,longitude],zoom_start=10)

for lat,lng,borough,neighborhood,postal_code in zip(toronto_data['Latitude'],toronto_data['Longitude'],toronto_data['Borough'],toronto_data['Neighborhood'],toronto_data['PostalCode']):
    label='{},{}'.format(postal_code,borough)
    label=folium.Popup(label,parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(toronto_map)

toronto_map

In [32]:
CLIENT_ID = 'CW5QX1Q0IVLAR5ORY53KAB2M5DPPR01FVB1GSMKE0BWNLRK5' # your Foursquare ID
CLIENT_SECRET = 'EQW0DAHV3FFE4DT22CBEBOX4UBJQLHVEFYWGNAKHVT0PAIQO' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version
LIMIT=100

# Explore Each Neighborhood's Venues in Karachi

### Create a function to extract the venues from each Neighborhood

In [33]:
def getNearbyVenues(names, latitudes, longitudes, radius=2000):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):  
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['id'],
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue ID','Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [34]:
toronto_venues = getNearbyVenues(names=toronto_data['Neighborhood'],
                                   latitudes=toronto_data['Latitude'],
                                   longitudes=toronto_data['Longitude']                                   
                                  )

FileNotFoundError: [Errno 2] File toronto_venues.csv does not exist: 'toronto_venues.csv'

# Size of Data Frame Toronto_Venues

In [26]:
print(toronto_venues.shape)
toronto_venues.head()

(7351, 8)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue ID,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Harbourfront,43.65426,-79.360636,54ea41ad498e9a11e9e13308,Roselle Desserts,43.653447,-79.362017,Bakery
1,Harbourfront,43.65426,-79.360636,53b8466a498e83df908c3f21,Tandem Coffee,43.653559,-79.361809,Coffee Shop
2,Harbourfront,43.65426,-79.360636,4ad4c05ef964a520bff620e3,The Distillery Historic District,43.650244,-79.359323,Historic Site
3,Harbourfront,43.65426,-79.360636,5612b1cc498e3dd742af0dc8,Impact Kitchen,43.656369,-79.356980,Restaurant
4,Harbourfront,43.65426,-79.360636,4c3e1eaa6faac9b66dc60d76,Distillery Sunday Market,43.650075,-79.361832,Farmers Market


## Exploring ratings, prices and tips of each venue

In [27]:
def append_str(re):
    try:
        rating = re['rating']
    except:
        rating = 0
    try:
        tips = re['tips']['count']
    except:
        tips = 0
    try:
        price = re['price']['message']
    except:
        price = "Not listed"

    return rating,tips,price

In [41]:
venues_data = []
for v_id in toronto_venues['Venue ID'].head(100):  #we'll explore only 500 venues because of API calls limitation
    venue_id = '4c50d7d7250dd13a12fa377c'
    url = 'https://api.foursquare.com/v2/venues/{}?client_id={}&client_secret={}&v={}'.format(
        venue_id,
        CLIENT_ID,
        CLIENT_SECRET,
        VERSION
    )
    
    results = requests.get(url).json()
    #print(results['response']['venue']['rating'],results['response']['venue']['tips']['count'])
    venues_data.append(append_str(results['response']['venue']['rating']))

venues_data

KeyError: 'venue'

In [37]:
total_venues = pd.DataFrame(venues_data, columns=['Ratings','Tips','Price'])
#T saved it to not utilize my calls again

total_venues.head()
total_venues.shape

(100, 3)

In [38]:
selected_venues=total_venues[(total_venues['Ratings'] !=0) & (total_venues['Price'] != "Not Listed") & (total_venues['Tips'] != 0.0)]
selected_venues

,Ratings,Tips,Price


# Sort venues by ratings

,Ratings,Tips,Price


# Visualizing the resulting clusters